In [1]:
import pandas as pd
import numpy as np
import requests
import socket
from ipwhois import IPWhois
import json
import websocket
from websocket import create_connection
import matplotlib.pyplot as plt
import folium
import csv

# Recover gateways

In [2]:
url = 'https://data.ripple.com/v2/gateways'
result = requests.get(url).json()

## Recover domain and currencies using gateway account

In [3]:
def get_domain(account):
    domain = ''
    
    url = 'https://data.ripple.com/v2/gateways/' + account
    result = requests.get(url).json()
    if 'domain' in result:
        domain =  result['domain']
    else:
        domain = np.nan
        
    if account == 'razqQKzJRdB4UxFPWf5NEpEG3WMkmwgcXA':
        domain = 'wg.iripplechina.com'
    
    if account == 'rcoef87SYMJ58NAFx7fNM5frVknmvHsvJ':
        domain = 'bpgrefining.com'
    
    if account == 'rsP3mgGb2tcYUrxiLFiHJiQXhsziegtwBc':
        domain = 'coinex.co.nz'
    
    curr_set = set()
    if 'accounts' in result:
        for acc in result['accounts']:
            for c in(acc['currencies']):
                curr_set.add(c)
    return domain, curr_set

## Use previously recovered domain to get IP addresses

In [4]:
def get_ip(domain):
    try:
        return socket.gethostbyname_ex(domain)[2]
    except:
        return np.nan

### Bring everything together

In [5]:
gateway_dict = {}
for currency in result:
    for elem in result[currency]:
        name = elem['name']
        account = elem['account']
        domain, curr_set = get_domain(account)
        ip_list = get_ip(domain)
        gateway_dict[account] = {'name' : name, 'domain' : domain, 'currencies': curr_set, 'ip': ip_list}

In [6]:
gateways_df = pd.DataFrame.from_dict(gateway_dict,orient='index')

In [7]:
# RippleSingapore and DotPayco and JustCoin and PayRoutes are closed
# https://twitter.com/RippleSingapore/status/787552615005556736
# https://forum.ripple.com/viewtopic.php?f=3&t=15668
# https://www.ccn.com/norwegian-bitcoin-exchange-justcoin-exits-gracefully-after-being-dropped-by-bank
# https://www.xrpchat.com/topic/3607-a-list-of-bankrupt-gateways-lets-make/
# I highly suspect Ripula and Ripple exchange tokyo to be closed. They have no activity since 2014 (https://developers.ripple.com/data-api-v2-tool.html#get-account-transaction-history)

gateways_df

,name,domain,currencies,ip
r3ADD8kXSUKHd6zTCKfnKT3zV9EZHjzp1S,RippleUnion,rippleunion.com,{CAD},[23.20.239.12]
r94s8px6kSw1uZ1MV98dhSRTvc6VMPoPcN,TokyoJPY,tokyojpy.com,{JPY},"[104.28.12.46, 104.28.13.46]"
r9Dr5xwkeLegBeXq6ujinjSBLQzQ1zQGjH,Ripple Singapore,ripplesingapore.com,"{XAU, SGD, XAG, USD}",NaN
r9ZFPSb1TFdnJwbTMYHvVwFK1bQPUCVNfJ,Ripple Exchange Tokyo,ripple-exchange.tokyo,{JPY},NaN
rB3gZey7VWHYRqJHLoHDEJXJ2pEPNieKiS,Mr. Exchange,mr-ripple.com,"{BTC, REP, STR, ETC, ADA, DOG, LTC, JPY, ETH, ...","[18.205.177.181, 52.202.40.211]"
rBycsjqxD8RVZP5zrrndiVtJwht7Z457A8,Ripula,ripula.co.uk,{GBP},NaN
rDAN8tzydyNfnNf2bfUQY6iR96UbpvNsze,Gatehub Fifth,gatehub.net,"{REP, ETC, ETH, BTC}","[104.31.65.177, 104.31.64.177]"
rG6FZ31hDHN1K5Dkbma3PSB5uVCuVVRzfn,Bitso,bitso.com,"{MXN, BTC}","[104.20.11.111, 104.20.12.111]"
rJHygWcTLVpSXkowott6kzgZU6viQSVYM1,Justcoin,justcoin.com,"{NOK, BTC, STR, EUR, LTC}",NaN
rJRi8WW24gt9X85PHAxfWNPCizMMhqUQwg,Digital Gate Japan,ripple-market.jp,{JPY},[202.172.28.118]


## Convert IP addresses to ASN

In [9]:
def get_as(x):
    if type(x) is float:
        return np.nan, np.nan
    as_list = set()
    gateways_countries = set()
    for ip in x:
        try:
            obj = IPWhois(ip)
            result = obj.lookup_whois()
            as_list.add(result['asn'])
            gateways_countries.add(result['asn_country_code'])
        except:
            print('ERROR with ' + ip)
    return as_list, next(iter(gateways_countries))

In [10]:
gateways_df['asn'], gateways_df['countries'] = zip(*gateways_df['ip'].apply(lambda x: get_as(x)))

## Use ASN to get latitude/longitude

In [11]:
countries = pd.read_csv('country.csv', delimiter=',')
country_dict = countries.set_index('ISO 3166 Country Code')[['Latitude','Longitude']].dropna().to_dict()

In [12]:
gateways_df['latitude'] = gateways_df['countries'].apply(lambda x: country_dict['Latitude'].get(x),np.nan)
gateways_df['longitude'] = gateways_df['countries'].apply(lambda x: country_dict['Longitude'].get(x,np.nan))

# Compute paths

## Generate payload for API request

In [29]:
def gen_command(sender,receiver,receiver_currency):
    test_json = {
          "id": 2,
          "command": "ripple_path_find",
          "source_account": sender,
          "destination_account": receiver,
          "destination_amount": {
            "currency": receiver_currency,
            "value": "0.01",
            "issuer": receiver
          }
        }
    return test_json

## Convert API response in list of paths

In [30]:
def extract_paths(result):
    paths = []
    for p in result:
        path = []
        for acc in (p['paths_computed']):
            for c in acc:
                if 'account' in c:
                    path.append(c['account'])
                if 'issuer' in c:
                    path.append(c['issuer'])
        paths.append(path)
    return paths

## Bring everything together

In [31]:
def get_paths(sender,receiver,receiver_currency):
    to_send = gen_command(sender,receiver,receiver_currency)
    websocket.enableTrace(False)
    ws = create_connection('wss://s2.ripple.com:443')

    ws.send(json.dumps(to_send))
    result = ws.recv()
    return extract_paths(json.loads(result)['result']['alternatives'])

In [32]:
gateways_df = gateways_df.dropna()
gateways_accounts = gateways_df.index
raw_links = []

## Write statistics for each requests
file = open("log_gateways.txt","w") 
 
for index_sender, sender in gateways_df.iterrows():
    path_found = 0
    for index_receiver, receiver in gateways_df.iterrows():
        if not sender.equals(receiver):
            for c in receiver['currencies']:
                file.write('Sender : {} - Receiver : {} - Currency : {}\n'.format(index_sender,index_receiver,c))
                
                paths = get_paths(index_sender,index_receiver,c)
                count_in = set()
                count_out = set()
                file.write('# Paths found : {}\n'.format(len(paths)))
                if (len(paths) > 0):
                    path_found += len(paths)
                    for path in paths:
                        raw_links.append(path)
                        if (len(path) > 0):
                            for acc in path:
                                if acc in gateways_accounts:
                                    count_in.add(acc)
                                else:
                                    count_out.add(acc)
                    
                    in_len = len(count_in)
                    out_len = len(count_out)
                    file.write('% of gateways : {:.0%} vs {:.0%} \n'.format(in_len/(in_len+out_len), out_len/(in_len+out_len)))              
    print('Receiver : {} - # Paths : {}'.format(index_sender,path_found))
file.close() 

Receiver : r3ADD8kXSUKHd6zTCKfnKT3zV9EZHjzp1S - # Paths : 8
Receiver : r94s8px6kSw1uZ1MV98dhSRTvc6VMPoPcN - # Paths : 135
Receiver : rB3gZey7VWHYRqJHLoHDEJXJ2pEPNieKiS - # Paths : 775
Receiver : rDAN8tzydyNfnNf2bfUQY6iR96UbpvNsze - # Paths : 95
Receiver : rG6FZ31hDHN1K5Dkbma3PSB5uVCuVVRzfn - # Paths : 145
Receiver : rJRi8WW24gt9X85PHAxfWNPCizMMhqUQwg - # Paths : 99
Receiver : rKiCet8SdvWxPXnAgYarFUXMh1zCPz432Y - # Paths : 444
Receiver : rKxKhXZCeSDsbkyB8DVgxpjy5AHubFkMFe - # Paths : 86
Receiver : rLEsXccBGNR3UPuPu2hUXPjziKC3qKSBun - # Paths : 317
Receiver : rMAz5ZnK73nyNUL4foAvaxdreczCkG3vA6 - # Paths : 148
Receiver : rMwjYedjc7qqtKYVLiAccJSmCwih4LnE2q - # Paths : 305
Receiver : rP5ShE8dGBH6hHtNvRESdMceen36XFBQmh - # Paths : 110
Receiver : rPDXxSZcuVL3ZWoyU82bcde3zwvmShkRyF - # Paths : 279
Receiver : rPxU6acYni7FcXzPCMeaPSwKcuS2GTtNVN - # Paths : 109
Receiver : rUkMKjQitpgAM5WTGk79xpjT38DEJY283d - # Paths : 126
Receiver : razqQKzJRdB4UxFPWf5NEpEG3WMkmwgcXA - # Paths : 277
Receiver : rc

Now that we recovered the data, we need to extract ASes and edges to draw a graph

# Create Graph

## Create edges

Create dictionary from gateway account to ASN

In [33]:
gate_to_as = gateways_df['asn'].dropna().apply(lambda x: next(iter(x))).to_dict()

Remove accounts in paths that are not gateways

In [34]:
gate_links = []
for l in raw_links:
    only_gate = [elem for elem in l if elem in gateways_accounts]
    gate_links.append(only_gate)

Extract edges between ASes

In [35]:
links = set()
for l in gate_links:
    for i in range(len(l)-1):
        source = gate_to_as[l[i]]
        dest = gate_to_as[l[i+1]]
        
        sources = source.split(' ')
        dests = dest.split(' ')
        for s in sources:
            for d in dests:
                if(s != d):
                    original = (s,d)
                    reverse = (d,s)
                    if (original not in links) and (reverse not in links):
                        links.add(original)

Save computed links into a file

In [36]:
with open('gateway_links.csv', 'w') as f:
    writer = csv.writer(f,lineterminator='\n')
    for tup in links:
        writer.writerow(tup)

## Create nodes

In [37]:
def sanitize(x):
    elem = next(iter(x))
    if ' ' in elem:
        return '38895'
    return elem

In [38]:
gateways_df['lat-lon'] = list(zip(gateways_df.latitude, gateways_df.longitude))
ases = gateways_df[['asn','lat-lon']].copy()
ases['asn'] = ases['asn'].apply(lambda x: sanitize(x))
ases = ases.set_index('asn').to_dict()['lat-lon']

## Use nodes ande edges to draw map

In [39]:
def generate_map(ases,edges):
    edges_position = set()
    for elem in edges:
        lat = ases[elem[0]]
        lon = ases[elem[1]]
        edges_position.add((lat,lon))
        
    # Make an empty map
    m = folium.Map(location=[20, 0], tiles="Mapbox Bright", zoom_start=2)

    # I can add marker one by one on the map
    for elem in ases.keys():
        folium.Marker(ases[elem], popup=elem).add_to(m)

    for elem in edges_position:
        folium.PolyLine(locations = elem, weight=1).add_to(m) 
    return m

In [40]:
generate_map(ases,links)

In [41]:
ases

{'14618': (38.0, -97.0),
 '13335': (38.0, -97.0),
 '37907': (36.0, 138.0),
 '38895': (38.0, -97.0),
 '60781': (52.5, 5.75),
 '9371': (36.0, 138.0),
 '16509': (38.0, -97.0),
 '46606': (38.0, -97.0),
 '26496': (38.0, -97.0),
 '4766': (37.0, 127.5),
 '45102': (38.0, -97.0),
 '57127': (46.0, 15.0),
 '17511': (36.0, 138.0),
 '53667': (38.0, -97.0),
 '19551': (38.0, -97.0)}

In [62]:
ases['13335'] = (40,-97)
ases['38895'] = (41,-99)
ases['16509'] = (42,-101)
ases['46606'] = (43,-103)
ases['26496'] = (44,-105)
ases['45102'] = (45,-107)
ases['53667'] = (46,-109)
ases['19551'] = (47,-111)

ases['17511'] = (39,140)
ases['9371'] = (41,142)

In [64]:
generate_map(ases,links)